<a href="https://colab.research.google.com/github/SHUDengwei/Pkmer-Docs/blob/main/05_%E5%AF%B9%E8%AF%9D%E5%BC%8F%E5%81%A5%E5%BA%B7%E5%8A%A9%E6%89%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模块5｜多轮对话与上下文管理 - 个性化健康顾问

**注意**：本文件原为 04，现在对应新课程结构的模块5

**核心技术**：Chat Session与上下文管理

**目标**：构建能"记住"对话历史的AI健康助手，提供连贯的个性化建议

---

## 本模块学习目标

在这个模块中，我们将：
1. ✅ 理解Chat Session的作用
2. ✅ 使用`model.start_chat()`创建对话
3. ✅ 维护和利用对话历史
4. ✅ 实现多轮连续对话
5. ✅ 提供基于上下文的个性化建议

---

## 为什么需要对话上下文？

### 没有上下文的问题：

```
用户：我最近总失眠
AI：建议早睡早起，保持规律作息

用户：具体几点睡比较好？
AI：一般建议10-11点睡 ❌ (AI忘记了用户有失眠问题)
```

###有上下文的优势：

```
用户：我最近总失眠
AI：了解了。你通常几点上床？[记住：用户失眠]

用户：12点左右
AI：[记住：失眠 + 12点睡] 入睡需要多久？

用户：至少1小时
AI：[记住：失眠 + 12点上床 + 1小时入睡]
   看来是入睡困难。白天有什么压力吗？

用户：工作压力挺大的
AI：[结合所有上下文] 看来是压力导致的入睡困难。
   建议：睡前1小时放下手机，尝试冥想...
```

---

---


## 1️⃣ 环境配置

In [ ]:
# 安装依赖
%pip install -q google-genai

In [ ]:
import os
from google import genai
from google.genai import types

# 如果在 Colab 中运行
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
except ImportError:
    # 如果在本地运行，从环境变量读取
    api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    raise ValueError("❌ 请先配置 GEMINI_API_KEY")

client = genai.Client(api_key=api_key)
print("✅ Gemini API 配置成功！")

✅ Gemini API 配置成功！


## 2️⃣ 对比：无上下文 vs 有上下文

先看看没有上下文管理时会发生什么

In [ ]:
# 第一轮对话（无状态）
response1 = client.models.generate_content(
    model='gemini-2.5-pro',
    contents="我最近总失眠，怎么办？简短的回答"
)
print("👤 用户：我最近总失眠，怎么办？简短的回答")
print(f"🤖 AI：{response1.text}\n")
print("="*50 + "\n")

# 第二轮对话（AI不记得第一轮）
response2 = client.models.generate_content(
    model='gemini-2.5-pro',
    contents="那我应该几点睡？简短的回答"
)
print("👤 用户：那我应该几点睡？简短的回答")
print(f"🤖 AI：{response2.text}\n")

print("❌ 问题：AI不记得用户有失眠问题，给出了通用建议")

👤 用户：我最近总失眠，怎么办？简短的回答
🤖 AI：失眠很困扰，可以试试以下方法：

1.  **规律作息**：每天定时睡觉和起床，即使是周末。
2.  **改善环境**：保持卧室黑暗、安静、凉爽。
3.  **睡前放松**：睡前1小时远离手机和电脑，可以洗个热水澡、听些轻音乐或进行深呼吸。
4.  **注意饮食**：下午和晚上避免摄入咖啡因（咖啡、茶、可乐），睡前不大量进食或饮酒。
5.  **白天运动**：白天进行适度运动，但避免在睡前3小时内剧烈运动。

**如果持续不见好转，严重影响生活，建议及时咨询医生。**


👤 用户：那我应该几点睡？简短的回答
🤖 AI：**从你的起床时间往前推算7.5或9个小时。**

例如，如果你早上7点起床，可以试试晚上10点或11点半睡觉。这样可以保证你完成5或6个完整的睡眠周期。

❌ 问题：AI不记得用户有失眠问题，给出了通用建议


## 3️⃣ 使用Chat Session管理上下文

使用`model.start_chat()`创建带有历史记录的对话


In [ ]:
# 创建Chat Session（使用空历史）
chat_history = [
    types.Content(role='user', parts=[types.Part(text="简单清晰的回答用户问题")]) # 初始 prompt
]

print("✅ Chat Session已创建")
print(f"   历史消息数：{len(chat_history)}")

✅ Chat Session已创建
   历史消息数：1


## 4️⃣ 示例1：多轮失眠咨询对话

AI会记住之前的对话，提供连贯的建议


In [ ]:
# 辅助函数：发送消息并更新历史
def send_message(message, history):
    # 添加用户消息到历史
    history.append(
        types.Content(role='user', parts=[types.Part(text=message)]))

    print(history)
    # 调用API
    response = client.models.generate_content(
        model='gemini-2.5-pro',
        contents=history
    )

    # 添加AI响应到历史
    history.append(types.Content(role='model', parts=[types.Part(text=response.text)]))

    return response

# 第1轮对话
print("【第1轮对话】")
response = send_message("我最近总失眠，怎么办？", chat_history)
print("👤 用户：我最近总失眠，怎么办？")
print(f"🤖 AI：{response.text}\n")
print("="*50 + "\n")

# 第2轮对话
print("【第2轮对话】")
response = send_message("我通常12点左右上床", chat_history)
print("👤 用户：我通常12点左右上床")
print(f"🤖 AI：{response.text}\n")
print("="*50 + "\n")

# 第3轮对话
print("【第3轮对话】")
response = send_message("入睡需要至少1小时", chat_history)
print("👤 用户：入睡需要至少1小时")
print(f"🤖 AI：{response.text}\n")
print("="*50 + "\n")

# 第4轮对话
print("【第4轮对话】")
response = send_message("工作压力确实挺大的", chat_history)
print("👤 用户：工作压力确实挺大的")
print(f"🤖 AI：{response.text}\n")
print("="*50 + "\n")

# 第5轮对话
print("【第5轮对话】")
response = send_message("有什么具体的改善建议吗？", chat_history)
print("👤 用户：有什么具体的改善建议吗？")
print(f"🤖 AI：{response.text}\n")

print("✅ AI记住了所有信息：")
print("   - 失眠问题")
print("   - 12点上床")
print("   - 入睡需要1小时")
print("   - 工作压力大")
print("   - 给出了针对性的建议")

【第1轮对话】
[Content(
  parts=[
    Part(
      text='简单清晰的回答用户问题'
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text='我最近总失眠，怎么办？'
    ),
  ],
  role='user'
)]
👤 用户：我最近总失眠，怎么办？
🤖 AI：好的，非常理解你被失眠困扰的感受。失眠确实很痛苦。

这里为你整理了一些简单清晰、可以立刻开始尝试的方法，希望能帮到你。

你可以把这些方法看作一个“睡眠工具箱”，尝试几种，找到最适合你的组合。

### 一、 睡前准备（营造睡觉的“仪式感”）

1.  **放松身体**：睡前1-2小时，洗个热水澡、泡个脚，或者用温毛巾热敷一下眼睛和脖子，让身体暖和放松下来。
2.  **远离电子产品**：至少在睡前1小时，放下手机、电脑、电视。屏幕的蓝光会抑制褪黑素（帮助你入睡的激素）的分泌。可以改成看会儿纸质书、听听舒缓的音乐或播客。
3.  **创造舒适环境**：把卧室弄得**暗一点、安静一点、凉快一点**。可以使用遮光窗帘、耳塞或白噪音机。
4.  **别在床上干别的事**：床只用来睡觉和亲密行为。不要在床上工作、玩手机或吃饭，让你的大脑把“床”和“睡觉”牢牢绑定在一起。

### 二、 白天习惯（调整你的生物钟）

1.  **作息规律**：每天尽量在**同一时间睡觉，同一时间起床**，即使是周末也要坚持。这是最有效的方法之一。
2.  **白天多晒太阳**：早上起床后，拉开窗帘，晒15-30分钟太阳。这能帮你校准生物钟，让身体知道“现在是白天，该清醒了”。
3.  **规律运动**：白天进行适度的运动（比如散步、慢跑、瑜伽）非常有帮助，但**不要在睡前3小时内剧烈运动**，否则会让身体太兴奋。
4.  **注意饮食**：
    *   **避免咖啡因和酒精**：下午之后尽量不喝咖啡、浓茶、可乐等含咖啡因的饮料。酒精虽然能让你很快睡着，但会严重破坏后半夜的睡眠质量。
    *   **睡前别吃太饱或太饿**：晚餐别吃太晚。如果睡前饿了，可以喝一小杯温牛奶或吃一根香蕉。

### 三、 心态调整（别跟“睡不着”这件事较劲）

1.  **放下“必须睡着”的压力**：越是强迫自己入睡，

## 5️⃣ 查看对话历史

Chat Session自动保存了所有对话


In [ ]:
print(f"📚 对话历史中有 {len(chat_history)} 条消息\n")

for i, message in enumerate(chat_history):
    role = "👤 用户" if message.role == "user" else "🤖 AI"
    content = message.parts[0].text
    # 只显示前50个字符
    preview = content[:50] + "..." if len(content) > 50 else content
    print(f"{i+1}. {role}: {preview}")

📚 对话历史中有 11 条消息

1. 👤 用户: 简单清晰的回答用户问题
2. 👤 用户: 我最近总失眠，怎么办？
3. 🤖 AI: 好的，非常理解你被失眠困扰的感受。失眠确实很痛苦。

这里为你整理了一些简单清晰、可以立刻开始尝试的...
4. 👤 用户: 我通常12点左右上床
5. 🤖 AI: 好的，了解了。12点左右上床，这个信息很重要。

**12点上床本身不是问题**，很多人都是这个作息...
6. 👤 用户: 入睡需要至少1小时
7. 🤖 AI: 好的，收到了这个非常具体的信息——“入睡需要至少1小时”。

这非常典型，也是失眠最折磨人的地方之一...
8. 👤 用户: 工作压力确实挺大的
9. 🤖 AI: 这一下子就说到了失眠最核心的点上。**工作压力大是导致入睡困难的最常见原因之一。**

我非常理解你...
10. 👤 用户: 有什么具体的改善建议吗？
11. 🤖 AI: 当然！我们把之前讨论的所有内容，整合成一个清晰、具体、可操作的**“三步改善计划”**。

你可以把...


## 6️⃣ 示例2：带有用户健康背景的对话

可以在创建Chat时注入用户的健康档案作为系统上下文


In [ ]:
# 小明的健康档案（来自前面模块的数据）
user_profile = """
用户档案：
- 姓名：小明
- 年龄：28岁
- 职业：互联网产品经理
- 健康目标：每天睡足8小时，每周运动150分钟
- 最近一周数据：
  * 平均睡眠：7.3小时（低于目标）
  * 运动情况：周一跑步30分钟，周三瑜伽45分钟，周五游泳40分钟（共115分钟，低于目标）
  * 饮食：周四吃了快餐，其他几天饮食健康
  * 压力水平：周二和周四压力很高
"""

# 创建带有用户档案的Chat
chat_with_context = [
    types.Content(
        role="user",
        parts=[types.Part(text=f"我是小明，这是我的健康档案：{user_profile}\n请作为我的健康顾问，帮助我改善健康状况。")]
    ),
    types.Content(
        role="model",
        parts=[types.Part(text="你好小明！我已经了解了你的健康档案。我看到你最近一周的睡眠和运动都没有达到目标，而且有几天压力较大。我会作为你的健康顾问，帮助你逐步改善这些方面。有什么具体想咨询的吗？")]
    )
]

print("✅ 已创建带有用户健康档案的Chat Session\n")
print("="*50 + "\n")

✅ 已创建带有用户健康档案的Chat Session




## 7️⃣ 基于档案的个性化对话

AI会结合用户的健康档案给出针对性建议

In [ ]:
# 对话1：询问睡眠改善
print("【对话1】")
response = send_message("怎么才能睡足8小时？", chat_with_context)
print("👤 小明：怎么才能睡足8小时？")
print(f"🤖 AI：{response.text}\n")
print("="*50 + "\n")

# 对话2：询问运动建议
print("【对话2】")
response = send_message("我还差35分钟运动时间，怎么补上？", chat_with_context)
print("👤 小明：我还差35分钟运动时间，怎么补上？")
print(f"🤖 AI：{response.text}\n")
print("="*50 + "\n")

# 对话3：压力管理
print("【对话3】")
response = send_message("工作压力大的时候应该怎么应对？", chat_with_context)
print("👤 小明：工作压力大的时候应该怎么应对？")
print(f"🤖 AI：{response.text}\n")

print("✅ 观察：AI的建议都基于小明的具体情况：")
print("   - 知道他的睡眠目标是8小时")
print("   - 知道他差35分钟才能达到运动目标")
print("   - 知道他周二和周四压力较大")

## 8️⃣ 实现交互式健康顾问

创建一个可以持续对话的助手

In [ ]:
def create_health_advisor(user_name="用户", user_info=""):
    """创建健康顾问Chat Session"""
    system_prompt = f"""你是一位专业的AI健康顾问，正在为{user_name}提供健康咨询服务。

角色定位：
- 专业：基于健康科学知识提供建议
- 关怀：理解用户的困难和挑战
- 责任：强调建议仅供参考，不替代专业医疗

{f'用户背景：{user_info}' if user_info else ''}

重要提醒：
⚠️ 始终强调你的建议仅供参考，严重健康问题需要就医
⚠️ 不做疾病诊断，只提供健康生活方式建议
"""

    chat_history = [
        types.Content(role="user", parts=[types.Part(text=system_prompt)]),
        types.Content(role="model", parts=[types.Part(text=f"你好{user_name}！我是你的AI健康顾问。我会基于健康科学知识为你提供建议，帮助你改善生活方式。需要提醒的是，我的建议仅供参考，严重健康问题请及时就医。有什么可以帮到你的吗？")])
    ]

    return chat_history

def chat_with_advisor(chat_history, max_turns=10):
    """与健康顾问进行对话"""
    print("💬 健康顾问对话（输入'退出'结束对话）\n")
    print("🤖 助手：你好！我是你的AI健康顾问，有什么可以帮到你的吗？\n")

    for turn in range(max_turns):
        user_input = input("👤 你：")

        if user_input.lower() in ['退出', 'quit', 'exit', 'q']:
            print("\n🤖 助手：再见！祝你健康！")
            break

        if not user_input.strip():
            continue

        response = send_message(user_input, chat_history)
        print(f"\n🤖 助手：{response.text}\n")

    print(f"\n📊 对话统计：共 {len(chat_history)} 条消息")

# 创建健康顾问
advisor_chat = create_health_advisor(
    user_name="小明",
    user_info=user_profile
)

print("✅ 健康顾问已创建！")
print("\n💡 提示：在实际使用时，取消下面代码的注释来进行交互式对话")
print("# chat_with_advisor(advisor_chat)")

✅ 健康顾问已创建！

💡 提示：在实际使用时，取消下面代码的注释来进行交互式对话
# chat_with_advisor(advisor_chat)
